In [ ]:
# Loading packages

# Visualization and Analysis
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import seaborn
import datetime as dt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# SQLAlchemy and ORM
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, and_

# Building Engine, Automapping Base, Staring Session
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)
inspector = inspect(engine)

# Viewing classes from automap
# Base.classes.keys()

In [ ]:
# Defining references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Printing table columns for reference
f"measurement table columns"
measurement_columns = inspector.get_columns('measurement')
for m_c in measurement_columns:
    print(m_c['name'], m_c["type"])
f"---"
f"station table columns"
station_columns = inspector.get_columns('station')
for s_c in station_columns:
    print(s_c['name'], s_c["type"])

In [ ]:
# Previewing table items
engine.execute('SELECT * FROM measurement LIMIT 10').fetchall()
engine.execute('SELECT * FROM station LIMIT 10').fetchall()

# Exploratory Climate Analysis

In [ ]:
# Notes: 
# - precipitation abbreviated as prcp and temperature 
# abbreviated as tobs like in measurement table
# - .label() is the same as SQL select as aliasing
# - Left "legacy code" as older alternate solutions
# - Referenced many external pages and left links in comments

# Figuring out the latest date and the year prior in the dataset
latest_date_query = str(session.query(Measurement.date).order_by(Measurement.date.desc()).first())
latest_date = latest_date_query[2:12]
f"The latest date in the dataset: {latest_date}"

year_before_latest_date_calc = str(int(latest_date[0:4]) - 1)
year_before_latest_date = year_before_latest_date_calc + latest_date[4:]
f"The year before the latest date in the dataset: {year_before_latest_date}"

# Retrieving last 12 months of prcp data
prcp_raw_points = session.query(Measurement.date.label('Date'),
                    Measurement.station.label('Station_ID'),
                    Measurement.prcp.label('Precipitation levels')).\
                    filter(Measurement.date.between(year_before_latest_date,latest_date)).\
                    order_by(Measurement.date)
prcp_df = pd.read_sql(prcp_raw_points.statement, prcp_raw_points.session.bind)

# Legacy code: Using list comprehensions to unpack the output of tuples
# dates = [prcp_raw_point[0] for prcp_raw_point in prcp_raw_points]
# station_names = [prcp_raw_point[1] for prcp_raw_point in prcp_raw_points]
# prcp_points = [prcp_raw_point[2] for prcp_raw_point in prcp_raw_points]
# prcp_df = pd.DataFrame(
#     {'Date': dates,
#      'Station': station_names,
#      'Precipitation levels': prcp_points})

# Setting Date as the index 
prcp_df.set_index('Date', inplace=True)

# Sorting by Date
prcp_df.sort_values(by='Date', ascending=True).head(10)
# prcp_df.tail(10)

In [ ]:
# # Plotting figure of prcp_df
plt.figure(figsize=(18,4))
plt.plot(prcp_df.index, prcp_df['Precipitation levels'])
plt.ylabel("Precipitation (inches)")
plt.xlabel("Date")
plt.title("Precipitation Levels 08/23/2016 - 08/23/2017")
plt.tight_layout()
plt.show()

# Tried to get the formatting to work on xticks, couldn't fix it in this case

In [ ]:
# prcp_df summary statistics
prcp_df.describe()

# Number of stations in dataset
HI_stations_count = session.query(Station).group_by(Station.station).count()
f"There are {HI_stations_count} weather stations in this dataset."

In [ ]:
# Listing stations in descending order of activity 
# in the whole dataset (via tobs count)
# Putting it into a dataframe for readability
StationActivity = session.query(Station.station.label('Station_ID'), 
                                Station.name.label('Station_Name'),
                                func.count(Measurement.tobs).label('Activity_Count')).\
                                filter(Station.station == Measurement.station).\
                                group_by(Measurement.station).\
                                order_by(func.count(Measurement.tobs).desc())
StationActivity_df = pd.read_sql(StationActivity.statement, StationActivity.session.bind)
StationActivity_df

f"The station with the most activity is Station ID: {StationActivity_df.at[0,'Station_ID']},"
f"at {StationActivity_df.at[0,'Station_Name']} with {StationActivity_df.at[0,'Activity_Count']} datapoints in the dataset."

In [ ]:
# Calculating the lowest temperature recorded, highest temperature recorded, 
# and average temperature from WAIHEE 837.5 in the whole dataset

Lowest_tobs_calc = session.query(func.min(Measurement.tobs)).\
            filter(Station.station == 'USC00519281').all()
Lowest_tobs_format = str(Lowest_tobs_calc[0])
Lowest_tobs = Lowest_tobs_format[1:3]
f"Lowest temperature recorded at Waihee: {Lowest_tobs} degrees"

Highest_tobs_calc = session.query(func.max(Measurement.tobs)).\
            filter(Station.station == 'USC00519281').all()
Highest_tobs_format = str(Highest_tobs_calc[0])
Highest_tobs = Highest_tobs_format[1:3]
f"Lowest temperature recorded at Waihee: {Highest_tobs} degrees"

Avg_tobs_calc = session.query(func.avg(Measurement.tobs)).\
            filter(Station.station == 'USC00519281').all()
Avg_tobs_format = str(Avg_tobs_calc[0])
Avg_tobs = Avg_tobs_format[1:3]
f"Average temperature recorded at Waihee: {Avg_tobs} degrees"

In [ ]:
# Retrieving last 12 months of tobs data from all stations
tobs_raw_points = session.query(Measurement.date.label('Date'),
                    Measurement.station.label('Station_ID'),
                    Measurement.tobs.label('Observed Temp')).\
                    filter(Measurement.date.between(year_before_latest_date,latest_date)).\
                    order_by(Measurement.date)

tobs_df = pd.read_sql(tobs_raw_points.statement, tobs_raw_points.session.bind)

# Setting Date as the index
tobs_df.set_index('Date', inplace=True)

# Sorting by Date
tobs_df.sort_values(by='Date', ascending=True).head(10)
# tobs_df.tail(10)

In [ ]:
# Retrieving last 12 months of tobs data from Waihee
Waihee_raw_points = session.query(Measurement.date.label('Date'),
                    Measurement.tobs.label('Observed Temp')).\
                    filter(Measurement.date.between(year_before_latest_date,latest_date)).\
                    filter(Measurement.station == 'USC00519281').\
                    order_by(Measurement.date)

# Making a dataframe and finding its shape to confirm there
# is around a year's worth of data
Waihee_df = pd.read_sql(Waihee_raw_points.statement, Waihee_raw_points.session.bind)
Waihee_df.shape 
Waihee_df.head(5)

In [ ]:
# Plotting a histogram for the last 12 months of tobs data from WAIHEE 837.5
tobs = [x[1] for x in Waihee_raw_points]
plt.hist(tobs, bins=12)
plt.ylabel("Frequency")
plt.xlabel("Observed Temperature")
plt.title("Observed Temperatures at Waihee 08/23/2016 - 08/23/2017")
#plt.savefig()
plt.show()

In [ ]:
# Using provided `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
    Returns:
        TMIN, TAVE, and TMAX
    """
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# Using the date range 2017-03-05 to 2017-03-17
trip = calc_temps('2017-03-05', '2017-03-17')[0]
f"For the duration of 3/5/17 to 3/17/17:"
f"Min temp: {trip[0]}"
f"Max temp: {trip[2]}"
f"Average temp: {trip[1]}"

In [ ]:
# Plotting a bar chart for the average temp calculated above
# with the "peak-to-peak" (tmax-tmin) value as an error barr

plt.figure(figsize=(2,6))
plt.title("Trip Average Temp")
plt.ylabel("Temperature (F)")
plt.bar(1, trip[1], yerr = (trip[2] - trip[0]), tick_label = "")
# plt.savefig("Outputs/Temperature Analysis.png")
plt.show()

In [ ]:
# Calculating rainfall per weather station for the dates used above
# Sorted in descending order

AllStationRainfall_During_Trip = session.query(Station.station.label('Station_ID'), 
                                Station.name.label('Station_Name'),
                                Station.latitude.label('Latitude'),
                                Station.longitude.label('Longitude'),
                                Station.elevation.label('Elevation'),
                                func.sum(Measurement.prcp).label('Precipitation Total')).\
                                filter(Station.station == Measurement.station).\
                                filter(Measurement.date.between('2017-03-05', '2017-03-17')).\
                                group_by(Measurement.station).\
                                order_by(func.sum(Measurement.prcp).desc())
AllStationRainfall_During_Trip_df = pd.read_sql(AllStationRainfall_During_Trip.statement, 
                                                AllStationRainfall_During_Trip.session.bind)
AllStationRainfall_During_Trip_df

In [ ]:
# Using provided query to calculate the daily normals for the date range
# (daily normals = averages for tmin, tmax, and tavg for all historic 
# data matching a specific month and day)
def daily_normals(date):
    """Daily Normals. 
    Args:
        date (str): A date string in the format '%m-%d'       
    Returns:
        A list of tuples containing the daily normals, tmin, tmax, and tavg   
    """   
    sel = [func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date)

# Using the date range 2017-03-05 to 2017-03-17
# Making a list of dates, based on
# https://stackoverflow.com/a/24637447
trip_dates = []
trip_daily_normals = []

start = dt.datetime.strptime("03-05", "%m-%d")
end = dt.datetime.strptime("03-17", "%m-%d")
date_generated = [start + dt.timedelta(days=x) for x in range(0, (end-start).days + 1)]

for date in date_generated:
    trip_dates.append(date.strftime("%m-%d"))

# Making a list of daily normals from the dates generated above
for date in trip_dates:
    trip_daily_normals.append(daily_normals(date)[0])

# Unpacking the tuples from the above list
trip_tmins = [trip_daily_normal[0] for trip_daily_normal in trip_daily_normals]
trip_tmaxs = [trip_daily_normal[1] for trip_daily_normal in trip_daily_normals]
trip_tavgs = [trip_daily_normal[2] for trip_daily_normal in trip_daily_normals]

# Making a dataframe with these results, making trip_dates the index
dailynorms_df = pd.DataFrame(
    {'Trip Dates': trip_dates,'Min Temp': trip_tmins,
     'Max Temp': trip_tmaxs, 'Avg Temp': trip_tavgs})

# Setting Date as the index
dailynorms_df.set_index('Trip Dates', inplace=True)

# Sorting by Date
dailynorms_df.sort_values(by='Trip Dates', ascending=True).head(10)

In [ ]:
# Plotting daily normals as an area plot with `stacked=False`
# Reference: https://stackoverflow.com/questions/34421434/pandas-dataframe-area-plot-stacked-false-not-working
x = np.arange(0,len(trip_dates))
dailynorms_df.plot(kind='area', stacked=False, alpha=0.5, colormap='Oranges', grid=True, 
                   title='Daily Normals 03-05 to 03-17')
plt.legend(loc='lower right')
plt.rc('grid', linestyle=":", linewidth=1, color='gray')
plt.xticks(x, trip_dates, rotation = 45)
plt.show()

## See Step 2 for Flask API.py [hyperlink here when done]
